# Data_manipulation

In [2]:
import json
import pandas as pd
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  port=3306,
  user="root",
  password="Password",
  database="Data_Manipulation"
)
cursor = mydb.cursor()

In [3]:
from pandas.io.json import json_normalize

with open('sample_data_for_assignment.json',encoding='utf8') as json_data:
    data = json.load(json_data)

columns=data['cols']

In [5]:
columns

['name',
 'phone',
 'email',
 'address',
 'region',
 'country',
 'list',
 'postalZip',
 'currency']

In [6]:
cursor.execute("CREATE TABLE json_to_sql_table (name VARCHAR(255), phone VARCHAR(45), email VARCHAR(100), \
                 address VARCHAR(255), region VARCHAR(100), country VARCHAR(45), list VARCHAR(20), postalZip VARCHAR(20),\
                 currency VARCHAR(20))")

In [7]:
for values in data['data']:
    sql = "INSERT INTO json_to_sql_table (name, phone, email, address, region, country, list, postalZip, currency) \
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (values[0], values[1], values[2], values[3], values[4], values[5], values[6], values[7], values[8])
    cursor.execute(sql, val)
    
mydb.commit()
    

In [9]:

cursor.execute(" SELECT COUNT(*) from json_to_sql_table")
for x in cursor:
  print(x)

(500,)


500 rows were inserted into data

In [55]:
# Reading sql table into aur dataframe
mysql_to_pd=pd.read_sql("SELECT * FROM json_to_sql_table",mydb)

In [57]:
mysql_to_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       500 non-null    object
 1   phone      500 non-null    object
 2   email      500 non-null    object
 3   address    500 non-null    object
 4   region     500 non-null    object
 5   country    500 non-null    object
 6   list       500 non-null    object
 7   postalZip  500 non-null    object
 8   currency   500 non-null    object
dtypes: object(9)
memory usage: 35.3+ KB


 Write a program to change all email address to have the format - abc@gmail.com (note the ending should be gmail.com)


In [58]:
mysql_to_pd.email=mysql_to_pd.email.apply(lambda x: x.split('@')[0]+'@gmail.com')
mysql_to_pd

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,1-251-583-4993,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242-403,$14.59
1,Brielle Davidson,1-632-583-7386,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,1-716-949-4663,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,85K 8W1,$36.45
3,Geoffrey Long,(116) 475-4598,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,(491) 267-4372,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68
...,...,...,...,...,...,...,...,...,...
495,Adrian Rodriquez,1-425-958-2315,non.quam.pellentesque@gmail.com,107-4830 Lobortis Avenue,Noord Brabant,Russian Federation,1,49343,$90.17
496,Sonia Lamb,1-814-767-3340,suscipit.est@gmail.com,8062 Lectus St.,Kurgan Oblast,Sweden,9,393741,$56.21
497,Kylee Silva,1-196-572-3426,nibh.lacinia@gmail.com,Ap #336-1000 Diam Street,Drenthe,Vietnam,5,70801,$63.16
498,Jeanette Todd,1-384-486-3954,nunc@gmail.com,Ap #680-5749 Neque Avenue,Chandigarh,Ireland,9,3765 TK,$43.01


### The values for "postalZip" doesn't have same type of data (some are string, some are int, some are alpha numeric). Write a program to convert all values to int. In case of alpha numeric remove the letters and keep only the numbers and convert it to int.


In [59]:
mysql_to_pd.postalZip

0      66242-403
1          18317
2        85K 8W1
3          28826
4         335049
         ...    
495        49343
496       393741
497        70801
498      3765 TK
499       651193
Name: postalZip, Length: 500, dtype: object

In [60]:
def string_to_num(string):
    numeric=''
    for x in string:
        if x.isnumeric():
            numeric=numeric+x
    return numeric


In [61]:
mysql_to_pd.postalZip=mysql_to_pd.postalZip.apply(lambda x: string_to_num(x))

In [62]:
mysql_to_pd.postalZip=mysql_to_pd.postalZip.astype('int64')
mysql_to_pd

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,1-251-583-4993,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242403,$14.59
1,Brielle Davidson,1-632-583-7386,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,1-716-949-4663,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,8581,$36.45
3,Geoffrey Long,(116) 475-4598,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,(491) 267-4372,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68
...,...,...,...,...,...,...,...,...,...
495,Adrian Rodriquez,1-425-958-2315,non.quam.pellentesque@gmail.com,107-4830 Lobortis Avenue,Noord Brabant,Russian Federation,1,49343,$90.17
496,Sonia Lamb,1-814-767-3340,suscipit.est@gmail.com,8062 Lectus St.,Kurgan Oblast,Sweden,9,393741,$56.21
497,Kylee Silva,1-196-572-3426,nibh.lacinia@gmail.com,Ap #336-1000 Diam Street,Drenthe,Vietnam,5,70801,$63.16
498,Jeanette Todd,1-384-486-3954,nunc@gmail.com,Ap #680-5749 Neque Avenue,Chandigarh,Ireland,9,3765,$43.01


In [63]:
mysql_to_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       500 non-null    object
 1   phone      500 non-null    object
 2   email      500 non-null    object
 3   address    500 non-null    object
 4   region     500 non-null    object
 5   country    500 non-null    object
 6   list       500 non-null    object
 7   postalZip  500 non-null    int64 
 8   currency   500 non-null    object
dtypes: int64(1), object(8)
memory usage: 35.3+ KB


### Write a separate function which take input data as values under the column name "phone" and processes it to return equivalent ASCII char of the value taken 2 at a time from left to right. Considering the value should be in between 65 to 99. If two-digit number is less than 65 replace it with O (O for Orange).

In [64]:
def converter(string):
    number=string_to_num(string)
    code=''
    j=len(number)
    if j%2 == 1:
        j=j-1
    for i in range(0,j,2):
        if int(number[i:i+2]) < 65:
            code=code+'O'
        else:
            code=code+chr(int(number[i:i+2]))
    return code


In [65]:
mysql_to_pd.phone=mysql_to_pd.phone.apply(lambda x: converter(x))
mysql_to_pd

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,OOOOc,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242403,$14.59
1,Brielle Davidson,OOOOO,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,OO^^B,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,8581,$36.45
3,Geoffrey Long,OOKOb,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,OOCOH,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68
...,...,...,...,...,...,...,...,...,...
495,Adrian Rodriquez,OO_RO,non.quam.pellentesque@gmail.com,107-4830 Lobortis Avenue,Noord Brabant,Russian Federation,1,49343,$90.17
496,Sonia Lamb,OOLIO,suscipit.est@gmail.com,8062 Lectus St.,Kurgan Oblast,Sweden,9,393741,$56.21
497,Kylee Silva,O`OOO,nibh.lacinia@gmail.com,Ap #336-1000 Diam Street,Drenthe,Vietnam,5,70801,$63.16
498,Jeanette Todd,OTOO_,nunc@gmail.com,Ap #680-5749 Neque Avenue,Chandigarh,Ireland,9,3765,$43.01


In [66]:
mysql_to_pd.rename(columns={'phone':'coded_phone_number'},inplace=True)
mysql_to_pd

,name,coded_phone_number,email,address,region,country,list,postalZip,currency
0,Winifred Branch,OOOOc,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242403,$14.59
1,Brielle Davidson,OOOOO,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,OO^^B,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,8581,$36.45
3,Geoffrey Long,OOKOb,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,OOCOH,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68
...,...,...,...,...,...,...,...,...,...
495,Adrian Rodriquez,OO_RO,non.quam.pellentesque@gmail.com,107-4830 Lobortis Avenue,Noord Brabant,Russian Federation,1,49343,$90.17
496,Sonia Lamb,OOLIO,suscipit.est@gmail.com,8062 Lectus St.,Kurgan Oblast,Sweden,9,393741,$56.21
497,Kylee Silva,O`OOO,nibh.lacinia@gmail.com,Ap #336-1000 Diam Street,Drenthe,Vietnam,5,70801,$63.16
498,Jeanette Todd,OTOO_,nunc@gmail.com,Ap #680-5749 Neque Avenue,Chandigarh,Ireland,9,3765,$43.01
